# Batch Legal – Model-Pipeline Week 1: Data Retrieval – Preprocessing – Modelling- Dictionaries – Plotly-Function for Streamlit




### Using all data

## Imports for the entire Notebook

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install --quiet -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.2 MB 5.0 MB/s 
     |████████████████████████████████| 181 kB 82.4 MB/s 
     |████████████████████████████████| 660 kB 70.0 MB/s 
     |████████████████████████████████| 10.1 MB 61.4 MB/s 
     |████████████████████████████████| 457 kB 93.8 MB/s 
     |████████████████████████████████| 42 kB 1.7 MB/s 
     |████████████████████████████████| 58 kB 7.7 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srs

In [3]:
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.1 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
!pip install spacy-lookups-data

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98.5 MB 1.3 MB/s 


In [3]:
#Imports

import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.collocations import *

import spacy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation


## Data Retrieval and Exploration

In [ ]:
#Jakob, you should enter your code here!

## Preprocessing

In [18]:
#Loading a lot of data from csv
"""CSV with 3146 documents"""

data = pd.read_csv("/content/drive/MyDrive/over_2800_docs_scraped.csv")

In [7]:
data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'title', 'cellar', 'date', 'dir_code',
       'dir_1', 'dir_2', 'dir_3', 'dir_4', 'dir_5', 'dir_6', 'Content'],
      dtype='object')

In [8]:
#Starting the actual Preprocessing
df_content = data.Content

In [9]:
df_content.head()

0    (1) Crowdfunding is increasingly an establishe...
1    (1) The Agreement on the withdrawal of the Uni...
2    (1) On 14 February 2020, the European Commissi...
3    (1) The horizontal principles set out in Artic...
4    (1) Every citizen of the Union has the fundame...
Name: Content, dtype: object

In [10]:
#Preproc-Function

def cleaning(sentence):
    
    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercasing 
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## removing numbers
    
    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## removing punctuation
    tokenized_sentence = word_tokenize(sentence) ## tokenizing 
    stop_words = set(stopwords.words('english')) ## defining stopwords
    tokenized_sentence_cleaned = [w for w in tokenized_sentence 
                                  if not w in stop_words] ## remove stopwords
    
    #tokenized_sentence_cleaned = [w for w in tokenized_sentence_cleaned if not w in ignore_list] COMMENTED IGNORE OUT!
    
    sentences = ' '.join(word for word in tokenized_sentence_cleaned)
    
    #spacy
    nlp = spacy.load('en_core_web_sm', disable=["tok2vec", "tagger", "parser", "attribute_ruler"])
    nlp.remove_pipe("lemmatizer")
    nlp.add_pipe("lemmatizer", config={"mode": "lookup"}).initialize()
    doc = nlp(sentences)
    lemmatized = " ".join([token.lemma_ for token in doc])
    
    return lemmatized

In [11]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [14]:
# Applying Cleaning Function

clean_txt = df_content.apply(cleaning)

In [ ]:
len(clean_txt)

2871

In [ ]:
type(clean_txt)

pandas.core.series.Series

In [38]:
clean_txt_df = pd.DataFrame(clean_txt)

In [40]:
#Save cleaned txt as csv

clean_txt_df.to_csv('/content/drive/MyDrive/2800_docs_clean_txt.csv')

In [15]:
all_content = ''.join(clean_txt)

In [16]:
from collections import Counter

# split() returns list of all the words in the string
split_it = all_content.split()

# Pass the split_it list to instance of Counter class.
Counters_found = Counter(split_it)
#print(Counters)

# most_common() produces k frequently encountered
# input values and their respective counts.
most_occur = Counters_found.most_common(100)


In [17]:
most_occur

[('shall', 130848),
 ('–', 117687),
 ('—', 104459),
 ('regulation', 101195),
 ('article', 100317),
 ('’', 56628),
 ('union', 55350),
 ('state', 52679),
 ('eu', 51565),
 ('member', 50829),
 ('commission', 50463),
 ('follow', 48055),
 ('b', 43097),
 ('use', 43044),
 ('‘', 42822),
 ('accordance', 41703),
 ('include', 39163),
 ('provide', 37390),
 ('point', 36896),
 ('refer', 36753),
 ('authority', 34999),
 ('information', 33711),
 ('product', 33245),
 ('c', 29944),
 ('annex', 29126),
 ('may', 28433),
 ('report', 28420),
 ('european', 27230),
 ('datum', 26167),
 ('measure', 25770),
 ('paragraph', 24868),
 ('set', 24258),
 ('little', 22180),
 ('period', 22089),
 ('apply', 22061),
 ('system', 21703),
 ('requirement', 21186),
 ('take', 21055),
 ('ec', 20545),
 ('risk', 20434),
 ('animal', 20311),
 ('country', 20012),
 ('market', 19599),
 ('concern', 19434),
 ('part', 19244),
 ('mean', 18967),
 ('institution', 18435),
 ('implement', 18024),
 ('financial', 17785),
 ('ensure', 17763),
 ('relevan

In [18]:
# list used to remove 10 most frequent words + 'european'

ignore_list = ['shall', 'regulation', 'article', 'union', 'state', 'eu', 'official',  'member', 'commission', 'commission', 'accordance', 'european']

In [19]:
#Function to get rid of these terms

def ignore(sentence):
  tokenized_sentence = word_tokenize(sentence) ## tokenizing 
  cleaned  = [w for w in tokenized_sentence if not w in ignore_list]
  sentence_cleaned = ' '.join(word for word in cleaned)
  return sentence_cleaned

In [20]:
#Applying function

txt_clean = clean_txt.apply(ignore)

In [21]:
txt_clean.head()

0    crowdfunding increasingly establish form alter...
1    agreement withdrawal unite kingdom great brita...
2    february ‘ ’ initiate antidumping investigatio...
3    horizontal principle set treaty teu treaty fun...
4    every citizen fundamental right move reside fr...
Name: Content, dtype: object

In [155]:
#Saving txt_clean

txt_clean.to_csv('/content/drive/MyDrive/preproc_txt.csv')

In [4]:
#Loading txt_clean

txt_clean = pd.read_csv('/content/drive/MyDrive/preproc_txt.csv')

In [8]:
txt_clean

,Unnamed: 0,Content
0,0,crowdfunding increasingly establish form alter...
1,1,agreement withdrawal unite kingdom great brita...
2,2,february ‘ ’ initiate antidumping investigatio...
3,3,horizontal principle set treaty teu treaty fun...
4,4,every citizen fundamental right move reside fr...
...,...,...
2866,2866,ec lay veterinary certification requirement im...
2867,2867,delegate specify single electronic report form...
2868,2868,experience gain insurance reinsurance undertak...
2869,2869,report consistent accurate comparable informat...


In [9]:
#Transforming Series in List

txt_clean = txt_clean.Content.tolist()

# Topic-Modelling with BERTopic




In [10]:
#PIP-installing BERTtopic

!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
from bertopic import BERTopic #BERTtopic-model: https://github.com/MaartenGr/BERTopic

In [12]:
#Training

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(txt_clean)

Batches:   0%|          | 0/90 [00:00<?, ?it/s]

2022-06-03 10:47:11,885 - BERTopic - Transformed documents to Embeddings
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
2022-06-03 10:47:29,431 - BERTopic - Reduced dimensionality
2022-06-03 10:47:30,090 - BERTopic - Clustered reduced embeddings


In [13]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,476,-1_design_follow_exceed_device
1,0,162,0_additive_fee_premixtures_substance
2,1,119,1_producer_price_export_import
3,2,99,2_crr_institution_exposure_report
4,3,95,3_animal_establishment_disease_consignment
...,...,...,...
68,67,11,67_amendment_specification_journal_publish
69,68,11,68_rice_husk_basmati_implement
70,69,11,69_biodiesel_us_taric_usa
71,70,11,70_ecb_loan_reserve_mfis


In [14]:
topic_model.visualize_topics()

In [15]:
topic_model.visualize_barchart()

In [16]:
topic_model.visualize_hierarchy()

In [69]:
topic_model.get_topic_freq()

,Topic,Count
0,-1,476
1,0,162
2,1,119
3,2,99
4,3,95
...,...,...
68,67,11
69,68,11
70,69,11
71,70,11


# Topic-Modelling Subdirectory »dir_3: Animal feedingstuffs«

In [19]:
#Changeing Content in DF

data.Content = txt_clean

In [20]:
#Sellecting only the documents characterized as »dir_3: Animal feedingstuffs«

df_selec = data.loc[data.dir_3 == "Animal feedingstuffs"]

In [21]:
animal_txt = df_selec.Content

In [22]:
#Transforming scalar to list

animal_txt = animal_txt.tolist()

### BERTopic-model

In [23]:
#Training on »dir_3: Animal feedingstuffs«

topic_model_animal_feed = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics_animal, probs = topic_model_animal_feed.fit_transform(animal_txt)

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

2022-06-03 10:48:19,454 - BERTopic - Transformed documents to Embeddings
2022-06-03 10:48:22,206 - BERTopic - Reduced dimensionality
2022-06-03 10:48:22,226 - BERTopic - Clustered reduced embeddings


In [24]:
topic_model_animal_feed.get_topic_info()

,Topic,Count,Name
0,0,167,0_additive_fee_use_animal
1,1,15,1_additive_saccharomyces_cerevisia_authorisation


In [25]:
topic_model_animal_feed.visualize_barchart()

In [26]:
topic_model_animal_feed.get_topics()

{0: [('additive', 0.09921865503808214),
  ('fee', 0.07879725121829234),
  ('use', 0.06425535275363926),
  ('animal', 0.049477477187290404),
  ('premixtures', 0.04554998570132615),
  ('substance', 0.043958368558410225),
  ('ec', 0.04390484736455651),
  ('content', 0.04273165801063127),
  ('measure', 0.037415026419314726),
  ('indicate', 0.03672709049666521)],
 1: [('additive', 0.15125959803557534),
  ('saccharomyces', 0.1329467467449407),
  ('cerevisia', 0.1329467467449407),
  ('authorisation', 0.10745933816645802),
  ('ec', 0.08943554669345186),
  ('fee', 0.08107568723494073),
  ('cncm', 0.07879572570711302),
  ('condition', 0.0670826613268744),
  ('preparation', 0.06685202080228446),
  ('use', 0.06592348877289757)]}

In [97]:
topic_model_animal_feed.get_topic(1)

[('additive', 0.15125959803557534),
 ('saccharomyces', 0.1329467467449407),
 ('cerevisia', 0.1329467467449407),
 ('authorisation', 0.10745933816645802),
 ('ec', 0.08943554669345186),
 ('fee', 0.08107568723494073),
 ('cncm', 0.07879572570711302),
 ('condition', 0.0670826613268744),
 ('preparation', 0.06685202080228446),
 ('use', 0.06592348877289757)]

### LDA-Model

In [ ]:
# bigram vectorization

vectorizer_n_gram = TfidfVectorizer(ngram_range = (1,1)) # BI-GRAMS
cleaned_vectorizer_n_gram = vectorizer_n_gram.fit_transform(animal_txt)

In [ ]:
df_animal = pd.DataFrame(cleaned_vectorizer_n_gram.toarray(), columns=vectorizer_n_gram.get_feature_names_out())

In [ ]:
#Topic model function from ML-10-lecture
def print_topics(model, vectorizer, top_words):
    for idx, topic in enumerate(model.components_):
        print("-"*20)
        print("Topic %d:" % (idx))
        print([(vectorizer_n_gram.get_feature_names_out()[i], round(topic[i],2))
                        for i in topic.argsort()[:-top_words - 1:-1]])

In [ ]:
# Instantiating the LDA 
n_components = 2
lda_model = LatentDirichletAllocation(n_components=n_components, max_iter = 100)

# Fitting the LDA on the vectorized documents
lda_model.fit(df_animal)

In [ ]:
print_topics(lda_model, cleaned_vectorizer_n_gram, top_words=20)

#Creating a function to speed up Modelling of Sub_Directories

In [131]:
def model_to_fig(data, name):
  df_selec = data.loc[data.dir_3 == name]
  txt = df_selec.Content
  txt = txt.tolist()
  
  model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
  topics, probs = model.fit_transform(txt)

  dictionary = {"Sub_dir Name:": name, "get_topics": model.get_topics(), "get_topic_freq": model.get_topic_freq(), "topics": topics}
  return dictionary


In [132]:
foodstuffs = model_to_fig(data, name = "Foodstuffs")

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2022-06-03 12:25:13,262 - BERTopic - Transformed documents to Embeddings
2022-06-03 12:25:16,161 - BERTopic - Reduced dimensionality
2022-06-03 12:25:16,177 - BERTopic - Clustered reduced embeddings


In [133]:
foodstuffs.keys()

dict_keys(['Sub_dir Name:', 'get_topics', 'get_topic_freq', 'topics'])

In [134]:
topic_freq = foodstuffs['get_topic_freq']

In [135]:
topic_freq

,Topic,Count
0,0,94
1,1,22
2,-1,2


In [136]:
get_topic = foodstuffs['get_topics']

In [137]:
import itertools
import numpy as np
from typing import List

import plotly.graph_objects as go
from plotly.subplots import make_subplots

def bert_bar(topic_freq, get_topic,
             topics: List[int] = None,
             top_n_topics: int = 10,
             n_words: int = 5,
             width: int = 250,
             height: int = 250) -> go.Figure:
  
  
  colors = itertools.cycle(["#D55E00", "#0072B2", "#CC79A7", "#E69F00", "#56B4E9", "#009E73", "#F0E442"])
  
  # Select topics based on top_n and topics args
  freq_df = topic_freq
  #freq_df = freq_df.loc[freq_df.Topic != -1, :]
  if topics is not None:
      topics = list(topics)
  elif top_n_topics is not None:
      topics = sorted(freq_df.Topic.to_list()[:top_n_topics])
  else:
      topics = sorted(freq_df.Topic.to_list()[0:6])
  
  print(topics)

  # Initialize figure
  subplot_titles = [f"Topic {topic}" for topic in topics]
  columns = 4
  rows = int(np.ceil(len(topics) / columns))
  fig = make_subplots(rows=rows,
                      cols=columns,
                      shared_xaxes=False,
                      horizontal_spacing=.1,
                      vertical_spacing=.4 / rows if rows > 1 else 0,
                      subplot_titles=subplot_titles)

  # Add barchart for each topic
  row = 1
  column = 1
  for topic in topics:
    words = [word + "  " for word, _ in get_topic[topic]][:n_words][::-1]
    scores = [score for _, score in get_topic[topic]][:n_words][::-1]
    
    fig.add_trace(
        go.Bar(x=scores,
                   y=words,
                   orientation='h',
                   marker_color=next(colors)),
        row=row, col=column)

    if column == columns:
        column = 1
        row += 1
    else:
        column += 1

  # Stylize graph
  fig.update_layout(
      template="plotly_white",
      showlegend=False,
      title={
            'text': "<b>Topic Word Scores",
            'x': .5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(
                size=22,
                color="Black")
      },
      width=width*4,
      height=height*rows if rows > 1 else height * 1.3,
      hoverlabel=dict(
          bgcolor="white",
          font_size=16,
          font_family="Rockwell"
      ),
  )

  fig.update_xaxes(showgrid=True)
  fig.update_yaxes(showgrid=True)

  return fig


In [138]:
bert_bar(topic_freq, get_topic)

[-1, 0, 1]


#Getting Dictionaries of all dir_3 Dics

In [144]:
animal_feed = model_to_fig(data, name = "Animal feedingstuffs")

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

2022-06-03 12:31:56,972 - BERTopic - Transformed documents to Embeddings
2022-06-03 12:32:00,074 - BERTopic - Reduced dimensionality
2022-06-03 12:32:00,092 - BERTopic - Clustered reduced embeddings


In [145]:
animal_zoo = model_to_fig(data, name = "Animal health and zootechnics")

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

2022-06-03 12:32:06,098 - BERTopic - Transformed documents to Embeddings
2022-06-03 12:32:08,895 - BERTopic - Reduced dimensionality
2022-06-03 12:32:08,917 - BERTopic - Clustered reduced embeddings


In [146]:
plant_healt = model_to_fig(data, name = "Plant health")

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

2022-06-03 12:32:14,475 - BERTopic - Transformed documents to Embeddings
2022-06-03 12:32:17,116 - BERTopic - Reduced dimensionality
2022-06-03 12:32:17,132 - BERTopic - Clustered reduced embeddings


In [151]:
milk_products = model_to_fig(data, name = "Milk products")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2022-06-03 12:40:12,259 - BERTopic - Transformed documents to Embeddings
2022-06-03 12:40:15,129 - BERTopic - Reduced dimensionality
2022-06-03 12:40:15,138 - BERTopic - Clustered reduced embeddings


In [152]:
tariff_classification = model_to_fig(data, name = "Tariff classification")

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

2022-06-03 12:40:17,580 - BERTopic - Transformed documents to Embeddings
2022-06-03 12:40:20,401 - BERTopic - Reduced dimensionality
2022-06-03 12:40:20,413 - BERTopic - Clustered reduced embeddings


In [153]:
customs_terr = model_to_fig(data, name = "Common customs territory")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2022-06-03 12:40:36,344 - BERTopic - Transformed documents to Embeddings
2022-06-03 12:40:38,907 - BERTopic - Reduced dimensionality
2022-06-03 12:40:38,915 - BERTopic - Clustered reduced embeddings


In [154]:
consumerr_inf = model_to_fig(data, name = "Consumer information, education and representation")

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

2022-06-03 12:40:51,888 - BERTopic - Transformed documents to Embeddings
2022-06-03 12:40:55,429 - BERTopic - Reduced dimensionality
2022-06-03 12:40:55,453 - BERTopic - Clustered reduced embeddings


In [155]:
topic_out = [animal_feed, animal_zoo, plant_healt, foodstuffs, milk_products, tariff_classification, customs_terr, consumerr_inf]

In [156]:
topics_df = pd.DataFrame(topic_out)

In [157]:
topics_df.head()

,Sub_dir Name:,get_topics,get_topic_freq,topics
0,Animal feedingstuffs,"{0: [('additive', 0.09921865503808214), ('fee'...",Topic Count 0 0 167 1 1 15,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,Animal health and zootechnics,"{-1: [('gb', 0.2583553209518617), ('kingdom', ...",Topic Count 0 0 132 1 1 4...,"[0, 0, 0, 0, 2, 2, 0, 0, 1, 2, 0, 0, 1, 0, 0, ..."
2,Plant health,"{0: [('substance', 0.08834932480946928), ('app...",Topic Count 0 0 91 1 1 3...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ..."
3,Foodstuffs,"{-1: [('perchlorate', 0.2418874002371451), ('p...",Topic Count 0 0 94 1 1 2...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Milk products,"{-1: [('product', 0.07774197933765593), ('quot...",Topic Count 0 -1 12,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]"


In [162]:
#Save as csv

topics_df.to_csv('/content/drive/MyDrive/sub_dir_topics.csv')